In [10]:
# pyodbc-5.1.0 llama-index==0.9.45.post1 arize-phoenix==2.2.1 pyvis diskcache-5.6.3 llama-cpp-python-0.2.76 matplotlib 

In [22]:
import pyodbc 
import pandas as pd

import os                                                                                                                                                                                                          
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path("/Users/hmwangila/Documents/Documents/datascience_projects/2024/crewagents/agents/src/crewai/env"))
pas=os.getenv("DB_PASS")

table='Customer Loyalty Program Management Processes and Systems Review'
query="""
SELECT /* +parallel*/ DISTINCT
	UPPER(a.[ProjectCode]) ProjectCode,
	a.[ProjectTitle],
	a.[Category6] AS [ProjectScore],
	[ActiveAuditPlanTitle] AS [AuditPlanTitle],
	a.[Category9] AS [ProjectQuarter],		
	RTRIM(LTRIM(REPLACE(UPPER(c.EntityTitle), 'DIVISION',''))) as Division,
	a.[Category4] AS [ProjectSource],	
	[CurrentProjectPhaseValue] AS [ProjectPhase],
	[ScheduledStartDate] AS [ProjectScheduledStartDate],
	[ScheduledEndDate] AS [ProjectScheduledEndDate],
	d.IssueTitle,
    d.Category4 RiskRating,
    d.IsReleased IssueReleased,
    d.ReleaseDate IssueReleaseDate,
	d.IssueState, 	
	e.RecommendationTitle,
	g.Text2 as ManagementAction,
	e.DueDate,
    e.ReviewDueDate,
	RTRIM(LTRIM(REPLACE(UPPER(e.Category2), 'DIVISION',''))) RecommendationDivision,
	e.RecommendationState,
	e.Category4 TrackingStatus,
	e.LastStatusUpdate,
	e.LastStatusUpdateUser,
	e.LastStatusUpdateDate,
    CASE
		WHEN CONVERT(DATE,e.DueDate) < CONVERT(DATE,GETDATE()) AND e.RecommendationState != 'Closed' THEN 'Overdue'
		-- WHEN e.DueDate IS NULL AND e.RecommendationState != 'Closed' THEN 'Overdue'
		-- WHEN e.DueDate IS NULL AND e.RecommendationState IS NULL THEN 'Overdue'
		WHEN FORMAT(CONVERT(DATE,e.DueDate),'yyyyMM') = FORMAT(GetDate(),'yyyyMM') AND e.RecommendationState != 'Closed'
            -- THEN 'Due End of ' + FORMAT(CONVERT(DATE,GETDATE()), 'MMMM')
            THEN 'Due End Month'
		WHEN e.RecommendationState = 'Closed' THEN 'Closed'
		ELSE 'Not Due'
	END AS OverdueDescription,
	CASE
		WHEN CONVERT(DATE,e.DueDate) < CONVERT(DATE,GETDATE()) AND e.RecommendationState != 'Closed' THEN 1
		-- WHEN e.DueDate IS NULL AND e.RecommendationState != 'Closed' THEN 1
		-- WHEN e.DueDate IS NULL AND e.RecommendationState IS NULL THEN 1
		WHEN e.RecommendationState = 'Closed' THEN 0
		ELSE 0
	END AS OverdueStatus,
	CASE
		WHEN CONVERT(DATE,e.DueDate) < CONVERT(DATE,GETDATE()) AND e.RecommendationState != 'Closed'
			THEN DATEDIFF(day,CONVERT(DATE,ISNULL(e.DueDate, GetDate())),CONVERT(DATE,GETDATE()))	
		ELSE 0
	END AS OverdueDays,
 
    -- d.CreatedUserName as Auditor
	-- e.LastNotStartedUser as Auditor
    f.AssignedUserName  as Auditor,
	pvt.[IssueOwner],
	pvt.[SeniorManager],
	pvt.HOD
    
FROM (select * from [TeamMateReporting].[dbo].[V_ReportApi_Project] where lower([ProjectTitle]) like '%Customer Loyalty Program Management Processes and Systems Review%') a

inner join [TeamMateReporting].[dbo].[V_ReportApi_ObjectRelationship] b on a.ProjectID = b.ProjectID
inner join [TeamMateReporting].[dbo].[V_ReportApi_Entity] c on b.[EntityID] = c.[EntityID]
LEFT join [TeamMateReporting].[dbo].[V_ReportApi_Issue] d on b.IssueID = d.IssueId
LEFT join [TeamMateReporting].[dbo].[V_ReportApi_Recommendation] e on b.RecommendationID = e.RecommendationId
LEFT JOIN [TeamMateReporting].[dbo].[V_ReportApi_RecommendationText] g on e.RecommendationId = g.RecommendationId
LEFT JOIN (SELECT * FROM (
SELECT SecurityAssignmentId, IssueId, RoleTitle, AssignedUserName,
RANK() OVER(PARTITION BY IssueId ORDER BY SecurityAssignmentId DESC) as preparer_rank
FROM [TeamMateReporting].[dbo].V_ReportApi_SecurityAssignment
WHERE 1=1
AND RoleTitle = 'Preparer'
and IssueId IS NOT NULL
) preps
WHERE 1=1
AND preparer_rank = 1) f on f.IssueId = d.IssueId
left join (
SELECT RecommendationId, RecommendationTitle,
[Business Contact] as [IssueOwner], [Business Reviewer] as [SeniorManager], [Business Observer] as [HOD]
FROM
(SELECT A.RecommendationId, A.RecommendationTitle,
   CONCAT(C.FirstName, ' ', C.LastName) StaffName,
   D.[StringResourceValue]  
  
  FROM [TeamMateReporting].[dbo].[V_ReportApi_Recommendation] a
  LEFT JOIN [TeamMateReporting].[dbo].[FACT_AssigneeRoleAssignment] B ON A.RecommendationId = B.ObjectID
  LEFT JOIN [TeamMateReporting].[dbo].[V_ReportApi_User] C ON C.UserID = B.AssigneeObjectID
  LEFT JOIN [TeamMateReporting].[dbo].[V_NG_List_SysSecRole] D ON D.ID = B.RoleLID
  where 1=1
  --and A.RecommendationTitle = 'Customers should be assigned distinct device serial numbers when loans are issued'
  AND (a.RecommendationState != 'Closed' or a.RecommendationState is null)
  ) AS SourceTable
PIVOT
(
Max(StaffName)
FOR [StringResourceValue] IN ([Business Contact], [Business Reviewer],[Business Observer])
) AS PivotTable
) as pvt on pvt.RecommendationId = e.RecommendationId
Where 1=1
-- and (a.[CurrentProjectPhaseValue] IN ('Implementation Tracking', 'Migrated Implementation Tracking','Finalised','Migrated Issued','Responses Accepted') or a.[CurrentProjectPhaseValue] is null)
-- AND (a.[Category4] != 'Special Assignment' or a.[Category4] is null)
-- AND UPPER([ProjectCode]) NOT LIKE '%SP%'
-- uncomment below
AND d.IsReleased = '1'
-- end of uncomment
-- AND e.RecommendationTitle IS NOT NULL
---AND (e.RecommendationState != 'Closed' or e.RecommendationState is null)"""


# query="""select distinct(a.[ProjectTitle]) from [TeamMateReporting].[dbo].[V_ReportApi_Project] a"""

# @tool

def number_to_ordinal(n):
    # Convert a number to its ordinal representation
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return str(n) + suffix
        
def query_tool(query: str) -> str:
    sentences = []
    flattened_list = []
    conn = pyodbc.connect("DRIVER={ODBC Driver 18 for SQL Server};"
                          "SERVER=172.29.250.142,1433;"
                          "DATABASE=TeamMateReporting;"
                          "UID=bigdata_user;"
                          f"PWD={pas};"
                          "Login Timeout=30000;Encrypt=no;")
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    result = cursor.fetchall()
    cursor.close()
    conn.close()

    df = pd.DataFrame.from_records(result, columns=columns)    

    return df






In [23]:
import nest_asyncio

nest_asyncio.apply()
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.query_engine.pandas import PandasInstructionParser
from llama_index.prompts import PromptTemplate
import pandas as pd

In [31]:

from llama_index import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
llm = LlamaCPP(
# optionally, you can set the path to a pre-downloaded model instead of model_url
model_path="/Users/hmwangila/Downloads/Phi-3-mini-128k-instruct.Q5_K_S.gguf",#Meta-Llama-3-8B.Q5_K_M.gguf",
temperature=0.7,
max_new_tokens=256,
# llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
context_window=3900,
# kwargs to pass to __call__()
generate_kwargs={},
# kwargs to pass to __init__()
# set to at least 1 to use GPU
model_kwargs={"n_gpu_layers": 1},
# transform inputs into Llama2 format
messages_to_prompt=messages_to_prompt,
completion_to_prompt=completion_to_prompt,
verbose=True,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /Users/hmwangila/Downloads/Phi-3-mini-128k-instruct.Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 131072
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.att

In [32]:
df=query_tool(query)
df.head()

,ProjectCode,ProjectTitle,ProjectScore,AuditPlanTitle,ProjectQuarter,Division,ProjectSource,ProjectPhase,ProjectScheduledStartDate,ProjectScheduledEndDate,...,LastStatusUpdate,LastStatusUpdateUser,LastStatusUpdateDate,OverdueDescription,OverdueStatus,OverdueDays,Auditor,IssueOwner,SeniorManager,HOD
0,3-FY23,Customer Loyalty Program Management Processes...,Requires attention (75%),FY23 Audit plan,Q1,CONSUMER BUSINESS,Annual Audit Plan,Finalised,2022-04-04,2022-06-10,...,Update from Lilian Nyambu- For the implementat...,Awuor Lourrine Juliet,2023-04-03 12:12:22.630,Closed,0,0,Awuor Lourrine Juliet,None,None,None
1,3-FY23,Customer Loyalty Program Management Processes...,Requires attention (75%),FY23 Audit plan,Q1,CONSUMER BUSINESS,Annual Audit Plan,Finalised,2022-04-04,2022-06-10,...,The large SME customers with fixed services we...,Mildred Olima,2023-03-27 07:56:57.980,Closed,0,0,Awuor Lourrine Juliet,None,None,None
2,3-FY23,Customer Loyalty Program Management Processes...,Requires attention (75%),FY23 Audit plan,Q1,CONSUMER BUSINESS,Annual Audit Plan,Finalised,2022-04-04,2022-06-10,...,The customer journey was reviewed and revised ...,Awuor Lourrine Juliet,2023-11-01 14:42:51.113,Closed,0,0,Mildred Olima,None,None,None
3,3-FY23,Customer Loyalty Program Management Processes...,Requires attention (75%),FY23 Audit plan,Q1,CONSUMER BUSINESS,Annual Audit Plan,Finalised,2022-04-04,2022-06-10,...,The churned customer accounts have been closed...,Mildred Olima,2022-09-18 18:46:23.717,Closed,0,0,Mildred Olima,None,None,None
4,3-FY23,Customer Loyalty Program Management Processes...,Requires attention (75%),FY23 Audit plan,Q1,CONSUMER BUSINESS,Annual Audit Plan,Finalised,2022-04-04,2022-06-10,...,•The new customer onboarding communication has...,Awuor Lourrine Juliet,2022-09-28 08:16:00.680,Closed,0,0,Awuor Lourrine Juliet,None,None,None


In [38]:
df.columns

Index(['ProjectCode', 'ProjectTitle', 'ProjectScore', 'AuditPlanTitle',
       'ProjectQuarter', 'Division', 'ProjectSource', 'ProjectPhase',
       'ProjectScheduledStartDate', 'ProjectScheduledEndDate', 'IssueTitle',
       'RiskRating', 'IssueReleased', 'IssueReleaseDate', 'IssueState',
       'RecommendationTitle', 'ManagementAction', 'DueDate', 'ReviewDueDate',
       'RecommendationDivision', 'RecommendationState', 'TrackingStatus',
       'LastStatusUpdate', 'LastStatusUpdateUser', 'LastStatusUpdateDate',
       'OverdueDescription', 'OverdueStatus', 'OverdueDays', 'Auditor',
       'IssueOwner', 'SeniorManager', 'HOD'],
      dtype='object')

In [92]:

print(str(llm.complete(f'can you share a list of only critical recommendations for higher level leadership from {df[["RecommendationTitle"]]}. \
Check if recommendations can be merged and merge them if they will refer to largely similar recommendations at a high level. \
Do not share any other explanation text or notes or anything else, only a list of the issues/merged recommendations and everything should be numbered, if its not numbered drop it')))

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =      14.54 ms /   166 runs   (    0.09 ms per token, 11416.78 tokens per second)
llama_print_timings: prompt eval time =    7968.66 ms /   188 tokens (   42.39 ms per token,    23.59 tokens per second)
llama_print_timings:        eval time =   10030.94 ms /   165 runs   (   60.79 ms per token,    16.45 tokens per second)
llama_print_timings:       total time =   18117.93 ms /   353 tokens


 1. Develop an enterprise loyalty program tailored for defending market share against competitors.

2. Implement a customer portal that allows for easy initiation of loyalty programs by customers.

3. Establish policies and mechanisms to ensure the timely expiry of loyalty values, thus preventing churn due to dormant accounts.

4. Revamp the brand's message and program for kindergartens to resonate with target demographics effectively.

5. Develop a systematic process to track and guarantee the successful delivery of solutions, ensuring customer satisfaction and retention.

6. Redesign the redemption process at 'bonga everywhere', making it more user-friendly and efficient for consumers.


In [93]:
print(str(llm.complete(f'can you share a list of only critical issues for higher level leadership from {df[["IssueTitle"]]}. \
Check if issues can be merged and merge them if they will refer to largely similar issue at a high level. \
Do not share any other explanation text or notes or anything else, only a list of the issues/merged issues and everything should be numbered, if its not numbered drop it')))

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =       8.70 ms /    99 runs   (    0.09 ms per token, 11374.08 tokens per second)
llama_print_timings: prompt eval time =    5913.85 ms /   181 tokens (   32.67 ms per token,    30.61 tokens per second)
llama_print_timings:        eval time =    5563.45 ms /    98 runs   (   56.77 ms per token,    17.61 tokens per second)
llama_print_timings:       total time =   11541.00 ms /   279 tokens


 1. Customer segments not adequately covered by loyalty programs

2. Enterprise customers lacking self-service options

3. Loyalty points for enterprise customers remaining unexpired

4. Low enrollment of new customers in loyalty programs

5. Payment issues related to full license fees for a system underdevelopment

6. Reconciliation discrepancies on merchandised bonga products


In [364]:
issues=str(llm.complete(f'can you share a list of only critical issues for higher level leadership from {df[["IssueTitle"]]}. \
Check if issues can be merged and merge them if they will refer to largely similar issue at a high level. \
Do not share any other explanation text or notes or anything else, only a list of the issues/merged issues and everything should be numbered, if its not numbered drop it'))
issues

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =      10.90 ms /    94 runs   (    0.12 ms per token,  8624.64 tokens per second)
llama_print_timings: prompt eval time =    8585.05 ms /    80 tokens (  107.31 ms per token,     9.32 tokens per second)
llama_print_timings:        eval time =   43037.85 ms /    93 runs   (  462.77 ms per token,     2.16 tokens per second)
llama_print_timings:       total time =   51855.21 ms /   173 tokens


' 1. Uncovered customer segments by loyalty program\n\n2. Lack of enterprise-specific loyalty features\n\n3. Premature expiration of loyalty points for customers\n\n4. Insufficient new customer enrollments in the loyalty program\n\n5. Full license fees not paid, undermining system integrity\n\n6. Discrepancies detected during merchandise-related reconciliations'

In [366]:
def extract_issues(text):
    # Split the text by newline characters and filter out empty strings
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Extract the issue descriptions by removing the numbering and trimming whitespace
    issues = []
    for line in lines:
        if line[0].isdigit() and '.' in line:
            issue = line.split('.', 1)[1].strip()
            issues.append(issue)
    
    return issues

issues=extract_issues(issues)
issues

['Uncovered customer segments by loyalty program',
 'Lack of enterprise-specific loyalty features',
 'Premature expiration of loyalty points for customers',
 'Insufficient new customer enrollments in the loyalty program',
 'Full license fees not paid, undermining system integrity',
 'Discrepancies detected during merchandise-related reconciliations']

In [133]:
project_name=df['ProjectTitle'].unique()[0]

In [131]:
date_report=pd.to_datetime('today').strftime('%Y-%m-%d')

In [117]:
df.columns

Index(['ProjectCode', 'ProjectTitle', 'ProjectScore', 'AuditPlanTitle',
       'ProjectQuarter', 'Division', 'ProjectSource', 'ProjectPhase',
       'ProjectScheduledStartDate', 'ProjectScheduledEndDate', 'IssueTitle',
       'RiskRating', 'IssueReleased', 'IssueReleaseDate', 'IssueState',
       'RecommendationTitle', 'ManagementAction', 'DueDate', 'ReviewDueDate',
       'RecommendationDivision', 'RecommendationState', 'TrackingStatus',
       'LastStatusUpdate', 'LastStatusUpdateUser', 'LastStatusUpdateDate',
       'OverdueDescription', 'OverdueStatus', 'OverdueDays', 'Auditor',
       'IssueOwner', 'SeniorManager', 'HOD'],
      dtype='object')

In [129]:
chief=f"CHIEF {df['Division'].unique()[0]}"

In [135]:
audit_code=df['ProjectCode'].unique()[0]

In [151]:


def generate_text(prompt, max_tokens=15000):
    response = llm.complete(
        prompt=prompt
    )
    return response.text.strip()


In [238]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

def create_chart():
    # Example chart creation using Matplotlib
    plt.figure(figsize=(6, 4))
    categories = ['A', 'B', 'C', 'D']
    values = [4, 7, 1, 8]
    plt.bar(categories, values, color='blue')
    plt.title('Example Chart')
    plt.xlabel('Categories')
    plt.ylabel('Values')
    chart_path = 'chart.png'
    plt.savefig(chart_path)
    plt.close()
    return chart_path
create_chart()

'chart.png'

In [315]:
# from docx import Document
# from docx.enum.text import WD_ALIGN_PARAGRAPH
# from docx.shared import Pt, RGBColor, Inches
# import matplotlib.pyplot as plt

# from docx.oxml import OxmlElement
# from docx.oxml.ns import qn

# def set_cell_shading(cell, shading_color):
#     """Set shading (background color) for a table cell."""
#     tc = cell._element
#     tcPr = tc.get_or_add_tcPr()
#     shd = OxmlElement('w:shd')
#     shd.set(qn('w:val'), 'clear')
#     shd.set(qn('w:color'), 'auto')
#     shd.set(qn('w:fill'), shading_color)
#     tcPr.append(shd)

# def highlight_row(table, row_idx, shading_color):
#     """Highlight a specific row in a table."""
#     row = table.rows[row_idx]
#     for cell in row.cells:
#         set_cell_shading(cell, shading_color)

# def load_template(template_path):
#     return Document(template_path)

# def format_run(run, font_size=None, font_color=None, bold=None, italic=None):
#     if font_size:
#         run.font.size = Pt(font_size)
#     if font_color:
#         run.font.color.rgb = RGBColor(*font_color)
#     if bold is not None:
#         run.bold = bold
#     if italic is not None:
#         run.italic = italic

# def replace_placeholder(paragraph, placeholder, content, font_size=None, font_color=None, bold=None, italic=None):
#     if placeholder in paragraph.text:
#         inline = paragraph.runs
#         for item in inline:
#             if placeholder in item.text:
#                 item.text = item.text.replace(placeholder, content)
#                 format_run(item, font_size, font_color, bold, italic)

# def populate_template(doc, data):
#     for paragraph in doc.paragraphs:
#         for key, value in data.items():
#             replace_placeholder(paragraph, key, value['text'], value.get('font_size'), value.get('font_color'), value.get('bold'), value.get('italic'))
#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for paragraph in cell.paragraphs:
#                     for key, value in data.items():
#                         replace_placeholder(paragraph, key, value['text'], value.get('font_size'), value.get('font_color'), value.get('bold'), value.get('italic'))
#     return doc

# def populate_table_with_placeholder(doc, placeholder, content):
#     """Populate table cells containing a specific placeholder with content."""
#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for paragraph in cell.paragraphs:
#                     replace_placeholder(paragraph, placeholder, content)


# # def add_table_of_contents(doc):
# #     toc_paragraph = doc.add_paragraph()
# #     run = toc_paragraph.add_run()
# #     fldChar = docx.oxml.OxmlElement('w:fldChar')
# #     fldChar.set(docx.oxml.ns.qn('w:fldCharType'), 'begin')
# #     instrText = docx.oxml.OxmlElement('w:instrText')
# #     instrText.set(docx.oxml.ns.qn('xml:space'), 'preserve')
# #     instrText.text = 'TOC \\o "1-3" \\h \\z \\u'
# #     fldChar2 = docx.oxml.OxmlElement('w:fldChar')
# #     fldChar2.set(docx.oxml.ns.qn('w:fldCharType'), 'separate')
# #     fldChar3 = docx.oxml.OxmlElement('w:fldChar')
# #     fldChar3.set(docx.oxml.ns.qn('w:fldCharType'), 'end')
# #     run._r.append(fldChar)
# #     run._r.append(instrText)
# #     run._r.append(fldChar2)
# #     run._r.append(fldChar3)

# def add_table_of_contents(doc):
#     # Create a new paragraph for the TOC
#     toc_paragraph = doc.add_paragraph()
#     run = toc_paragraph.add_run()
    
#     # Create the field begin element
#     fldChar1 = OxmlElement('w:fldChar')
#     fldChar1.set(qn('w:fldCharType'), 'begin')
    
#     # Create the field instruction element
#     instrText = OxmlElement('w:instrText')
#     instrText.set(qn('xml:space'), 'preserve')
#     instrText.text = 'TOC \\o "1-3" \\h \\z \\u'
    
#     # Create the field separate element
#     fldChar2 = OxmlElement('w:fldChar')
#     fldChar2.set(qn('w:fldCharType'), 'separate')
    
#     # Create the field end element
#     fldChar3 = OxmlElement('w:fldChar')
#     fldChar3.set(qn('w:fldCharType'), 'end')
    
#     # Add the elements to the run
#     run._r.append(fldChar1)
#     run._r.append(instrText)
#     run._r.append(fldChar2)
#     run._r.append(fldChar3)
# def save_document(doc, output_path):
#     doc.save(output_path)

# def create_chart(chart_number):
#     # Example chart creation using Matplotlib
#     plt.figure(figsize=(4, 3))  # Default size, can be adjusted for specific charts
#     categories = ['A', 'B', 'C', 'D']
#     if chart_number == 1:
#         values = [4, 7, 1, 8]
#         plt.bar(categories, values, color='blue')
#         plt.title('Summary of Observations')
#     elif chart_number == 2:
#         values = [3, 6, 2, 9]
#         plt.bar(categories, values, color='green')
#         plt.title('Distribution of Observations per Scope Area')
#     elif chart_number == 3:
#         values = [5, 2, 3, 7]
#         plt.bar(categories, values, color='red')
#         plt.title('Root Cause Analysis of Findings')
#     plt.xlabel('Categories')
#     plt.ylabel('Values')
#     chart_path = f'chart_{chart_number}.png'
#     plt.savefig(chart_path)
#     plt.close()
#     return chart_path

# def insert_charts_side_by_side(doc, chart_paths, sizes, placeholder):
#     for paragraph in doc.paragraphs:
#         if placeholder in paragraph.text:
#             p = paragraph._element
#             p.clear()  # Clear the placeholder text

#             table = doc.add_table(rows=1, cols=len(chart_paths))
#             for i, chart_path in enumerate(chart_paths):
#                 cell = table.cell(0, i)
#                 run = cell.paragraphs[0].add_run()
#                 width, height = sizes[i]
#                 run.add_picture(chart_path, width=Inches(width), height=Inches(height))
#                 cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

#             p.addnext(table._element)  # Insert the table after the paragraph
#             return


# def main():
#     template_path = 'Audit Report Template.docx'  # Path to the provided template
#     output_path = 'Populated_Audit_Report.docx'
    
#     # Generate content for various sections
#     title = project_name.strip()
#     background = "Provide background information for the audit report."
#     objectives = "State the overall objectives of the audit."
#     scope = "Describe the scope of the audit."
#     opinion = "Provide an audit opinion."
#     executive_summary = "Write an executive summary with key highlights."
#     detailed_issues = "Describe detailed improvement opportunities and issues."
#     chief_name_title = "John Doe – Chief Audit Executive"

#     # Data to populate the template
#     #000- black
#     #00255 blue
#     data = {
#         'Audit Code':{'text':audit_code,'font_size': 18, 'font_color': (255, 255, 255), 'bold': True, 'italic': False}, 
#         'Project Title': {'text': title, 'font_size': 28, 'font_color': (255, 255, 255), 'bold': True, 'italic': False},
#         'XXXX': {'text': background, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
#         'Overall objectives': {'text': objectives, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
#         'Scope': {'text': scope, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
#         'Opinion and grading': {'text': opinion, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
#         'EXECUTIVE SUMMARY: Key highlights': {'text': executive_summary, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
#         'Description of Issue 1.': {'text': detailed_issues, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
#         'Report_Date': {'text': date_report, 'font_size': 18, 'font_color': (255, 255, 255), 'bold': False, 'italic': False},
#         'Chief Name':{'text': chief, 'font_size': 12.5, 'font_color': (255, 255, 255), 'bold': False, 'italic': False},
#         'Issue_things':{'text': 'kajgfkalgFJKGKLADFGligfiuEGFUWIEG', 'font_size': 12.5, 'font_color': (255, 255, 255), 'bold': False, 'italic': False}
#     }
    
#     # Load, populate and add TOC to the document
#     doc = load_template(template_path)
#     doc = populate_template(doc, data)
#     add_table_of_contents(doc)
    
#     # Create charts
#     chart_path_1 = create_chart(1)
#     chart_path_2 = create_chart(2)
#     chart_path_3 = create_chart(3)
    
#     # Define chart sizes (width, height in inches)
#     sizes = [(3, 2), (4, 2), (2, 2)]  # Chart 1: 3x2, Chart 2: 4x2, Chart 3: 2x2
    
#     # Insert charts side-by-side in a table
#     insert_charts_side_by_side(doc, [chart_path_1, chart_path_2, chart_path_3], sizes, 'Chart1')

#     # Find the table to modify (assuming the first table here)
#     table = doc.tables[2]
    
#     # Highlight the third row (index 2)
#     highlight_row(table, 2, '#FF0000')  # Yellow highlight (hex color code)

#         # Placeholder and content to replace
#     placeholder = 'ISSUES'
#     content = 'New issue content that replaces the placeholder text.'
    
#     # Populate the placeholder in table cells
#     populate_table_with_placeholder(doc, placeholder, content)
#     populate_table_with_placeholder(doc, 'RISKS', content)
#     populate_table_with_placeholder(doc, 'RATINGS', 'MEDIUM')
#     # Save the document
#     save_document(doc, output_path)

# if __name__ == "__main__":
#     main()


In [373]:
def replace_placeholder_in_paragraph(paragraph, placeholders):
    for run in paragraph.runs:
        for key, value in placeholders.items():
            if key in run.text:
                run.text = run.text.replace(key, value)

def replace_placeholders_in_table(table, placeholders):
    """Replace placeholders in a table with item-specific data."""
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                replace_placeholder_in_paragraph(paragraph, placeholders)

def find_paragraph_before_table(doc, table):
    """Find the paragraph that immediately precedes the specified table."""
    for paragraph in doc.paragraphs:
        if paragraph._element.getnext() is table._element:
            return paragraph
    return None

def clone_section_with_content(template_paragraph, template_table, placeholders):
    """Clone the title and table, then replace placeholders."""
    cloned_paragraph = deepcopy(template_paragraph)
    replace_placeholder_in_paragraph(cloned_paragraph, placeholders)
    
    cloned_table = deepcopy(template_table)
    replace_placeholders_in_table(cloned_table, placeholders)
    
    return cloned_paragraph, cloned_table

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt, RGBColor, Inches
import matplotlib.pyplot as plt

from docx.oxml import OxmlElement
from docx.oxml.ns import qn

def set_cell_shading(cell, shading_color):
    """Set shading (background color) for a table cell."""
    tc = cell._element
    tcPr = tc.get_or_add_tcPr()
    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), shading_color)
    tcPr.append(shd)

def highlight_row(table, row_idx, shading_color):
    """Highlight a specific row in a table."""
    row = table.rows[row_idx]
    for cell in row.cells:
        set_cell_shading(cell, shading_color)

def load_template(template_path):
    return Document(template_path)

def format_run(run, font_size=None, font_color=None, bold=None, italic=None):
    if font_size:
        run.font.size = Pt(font_size)
    if font_color:
        run.font.color.rgb = RGBColor(*font_color)
    if bold is not None:
        run.bold = bold
    if italic is not None:
        run.italic = italic

def replace_placeholder(paragraph, placeholder, content, font_size=None, font_color=None, bold=None, italic=None):
    if placeholder in paragraph.text:
        inline = paragraph.runs
        for item in inline:
            if placeholder in item.text:
                item.text = item.text.replace(placeholder, content)
                format_run(item, font_size, font_color, bold, italic)

def populate_template(doc, data):
    for paragraph in doc.paragraphs:
        for key, value in data.items():
            replace_placeholder(paragraph, key, value['text'], value.get('font_size'), value.get('font_color'), value.get('bold'), value.get('italic'))
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for key, value in data.items():
                        replace_placeholder(paragraph, key, value['text'], value.get('font_size'), value.get('font_color'), value.get('bold'), value.get('italic'))
    return doc

def populate_table_with_placeholder(doc, placeholder, content):
    """Populate table cells containing a specific placeholder with content."""
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    replace_placeholder(paragraph, placeholder, content)



def add_table_of_contents(doc):
    # Create a new paragraph for the TOC
    toc_paragraph = doc.add_paragraph()
    run = toc_paragraph.add_run()
    
    # Create the field begin element
    fldChar1 = OxmlElement('w:fldChar')
    fldChar1.set(qn('w:fldCharType'), 'begin')
    
    # Create the field instruction element
    instrText = OxmlElement('w:instrText')
    instrText.set(qn('xml:space'), 'preserve')
    instrText.text = 'TOC \\o "1-3" \\h \\z \\u'
    
    # Create the field separate element
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'separate')
    
    # Create the field end element
    fldChar3 = OxmlElement('w:fldChar')
    fldChar3.set(qn('w:fldCharType'), 'end')
    
    # Add the elements to the run
    run._r.append(fldChar1)
    run._r.append(instrText)
    run._r.append(fldChar2)
    run._r.append(fldChar3)
def save_document(doc, output_path):
    doc.save(output_path)

def create_chart(chart_number):
    # Example chart creation using Matplotlib
    plt.figure(figsize=(4, 3))  # Default size, can be adjusted for specific charts
    categories = ['A', 'B', 'C', 'D']
    if chart_number == 1:
        values = [4, 7, 1, 8]
        plt.bar(categories, values, color='blue')
        plt.title('Summary of Observations')
    elif chart_number == 2:
        values = [3, 6, 2, 9]
        plt.bar(categories, values, color='green')
        plt.title('Distribution of Observations per Scope Area')
    elif chart_number == 3:
        values = [5, 2, 3, 7]
        plt.bar(categories, values, color='red')
        plt.title('Root Cause Analysis of Findings')
    plt.xlabel('Categories')
    plt.ylabel('Values')
    chart_path = f'chart_{chart_number}.png'
    plt.savefig(chart_path)
    plt.close()
    return chart_path

def insert_charts_side_by_side(doc, chart_paths, sizes, placeholder):
    for paragraph in doc.paragraphs:
        if placeholder in paragraph.text:
            p = paragraph._element
            p.clear()  # Clear the placeholder text

            table = doc.add_table(rows=1, cols=len(chart_paths))
            for i, chart_path in enumerate(chart_paths):
                cell = table.cell(0, i)
                run = cell.paragraphs[0].add_run()
                width, height = sizes[i]
                run.add_picture(chart_path, width=Inches(width), height=Inches(height))
                cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

            p.addnext(table._element)  # Insert the table after the paragraph
            return


def main():
    template_path = 'Audit Report Template.docx'  # Path to the provided template
    output_path = 'Populated_Audit_Report.docx'
    
    # Generate content for various sections
    title = project_name.strip()
    background = "Provide background information for the audit report."
    objectives = "State the overall objectives of the audit."
    scope = "Describe the scope of the audit."
    opinion = "Provide an audit opinion."
    executive_summary = "Write an executive summary with key highlights."
    detailed_issues = "Describe detailed improvement opportunities and issues."
    chief_name_title = "John Doe – Chief Audit Executive"

    # Data to populate the template
    #000- black
    #00255 blue
    data = {
        'Audit Code':{'text':audit_code,'font_size': 18, 'font_color': (255, 255, 255), 'bold': True, 'italic': False}, 
        'Project Title': {'text': title, 'font_size': 28, 'font_color': (255, 255, 255), 'bold': True, 'italic': False},
        'XXXX': {'text': background, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
        'Overall objectives': {'text': objectives, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
        'Scope': {'text': scope, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
        'Opinion and grading': {'text': opinion, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
        'EXECUTIVE SUMMARY: Key highlights': {'text': executive_summary, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
        'Description of Issue 1.': {'text': detailed_issues, 'font_size': 12, 'font_color': (0, 0, 0), 'bold': False, 'italic': False},
        'Report_Date': {'text': date_report, 'font_size': 18, 'font_color': (255, 255, 255), 'bold': False, 'italic': False},
        'Chief Name':{'text': chief, 'font_size': 12.5, 'font_color': (255, 255, 255), 'bold': False, 'italic': False}
    }
    
    # Load, populate and add TOC to the document
    doc = load_template(template_path)

    
    # Create charts
    chart_path_1 = create_chart(1)
    chart_path_2 = create_chart(2)
    chart_path_3 = create_chart(3)
    
    # Define chart sizes (width, height in inches)
    sizes = [(2, 2), (4, 2), (2, 2)]  # Chart 1: 2x2, Chart 2: 4x2, Chart 3: 2x2
    
    # Insert charts side-by-side in a table
    insert_charts_side_by_side(doc, [chart_path_1, chart_path_2, chart_path_3], sizes, 'Chart1')

    # Find the table to modify (assuming the first table here)
    table = doc.tables[2]
    
    # Highlight the third row (index 2)
    highlight_row(table, 2, '#FF0000')  # Yellow highlight (hex color code)

        # Placeholder and content to replace
    placeholder = 'ISSUES'
    placeholder1 = 'RISKS'
    placeholder2 = 'RATINGS'
    placeholder3='RECOM'
    content = 'New issue content that replaces the placeholder text.'

      # Example issues data
    # issues = [
    #     {placeholder: "Issue description for no campaigns", "Description_of_Issue": "Description of Issue for no_campaigns",placeholder1: "no_campaign",placeholder2:"no_campaign_rating",placeholder3:"recoms_no_campaign"},
    #     {placeholder: "Issue description for poor engagement", "Description_of_Issue": "Description of Issue for poor_engagement",placeholder1: "poor_engagement",placeholder2:"poor_engagement_rating",placeholder3:"recoms_poor_engag"},
    #     {placeholder: "Issue description for low conversion", "Description_of_Issue": "Description of Issue for low_conversion",placeholder1: "low_conversion",placeholder2:"low_conversion_rating",placeholder3:"recoms_low_conversion"}
    # ]
    issues1=[]
    for i in issues:
        issues1.append({placeholder: i, "Description_of_Issue": f"Description of Issue for {i}",\
             placeholder1: f"risk {i}",placeholder2:f"{i}_rating",placeholder3:f"{i}_recom"})
    
    # Locate the template table (assuming it's the first table in the document)
    template_table_index = 4  # Adjust this index based on where your template table is located
    template_table = doc.tables[template_table_index]

    # Locate the paragraph immediately preceding the table
    template_paragraph = find_paragraph_before_table(doc, template_table)

    if template_paragraph is None:
        print("No paragraph found associated with the specified table.")
        return

    # Remove the original template elements
    doc.element.body.remove(template_table._element)
    doc.element.body.remove(template_paragraph._element)
    
    # Clone and fill the section for each issue
    for issue in issues1:
        placeholders = {
            "Description of Issue": issue["Description_of_Issue"],
            placeholder: issue[placeholder],
            placeholder1: issue[placeholder1],
            placeholder2: issue[placeholder2],
            placeholder3: issue[placeholder3]
        }
        
        cloned_paragraph, cloned_table = clone_section_with_content(template_paragraph, template_table, placeholders)
        
        # Append the cloned elements to the document
        doc.element.body.append(cloned_paragraph._element)
        doc.element.body.append(cloned_table._element)
        
        # Add a paragraph to separate sections
        doc.add_paragraph()

    
    # Populate the placeholder in table cells
    populate_table_with_placeholder(doc, 'Issue description for no campaigns', 'trail of stuff')
    # populate_table_with_placeholder(doc, 'RISKS', content)
    # populate_table_with_placeholder(doc, 'RATINGS', 'MEDIUM')

    doc = populate_template(doc, data)
    add_table_of_contents(doc)
    # Save the document
    save_document(doc, output_path)

if __name__ == "__main__":
    main()


In [33]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)
llm = llm

Looks like this: input query_str -> pandas_prompt -> llm1 -> pandas_output_parser -> response_synthesis_prompt -> llm2

Additional connections to response_synthesis_prompt: llm1 -> pandas_instructions, and pandas_output_parser -> pandas_output.

In [34]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")

In [39]:
response = qp.run(
    query_str="What are all the IssueTitle raised in the audit table?",
)

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?


> Running module input with input: 
query_str: What are all the IssueTitle raised in the audit table?

> Running module pandas_prompt with input: 
query_str: What are all the IssueTitle raised in the audit table?

> Running module llm1 with input: 
prompt: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  ProjectCode                                       ProjectTitle  \
0   ...



Llama.generate: prefix-match hit

llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =       1.72 ms /    11 runs   (    0.16 ms per token,  6406.52 tokens per second)
llama_print_timings: prompt eval time =   23834.60 ms /  1076 tokens (   22.15 ms per token,    45.14 tokens per second)
llama_print_timings:        eval time =    8686.61 ms /    10 runs   (  868.66 ms per token,     1.15 tokens per second)
llama_print_timings:       total time =   32554.36 ms /  1086 tokens
llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit


> Running module pandas_output_parser with input: 
input:  df['IssueTitle'].tolist()

> Running module response_synthesis_prompt with input: 
query_str: What are all the IssueTitle raised in the audit table?
pandas_instructions:  df['IssueTitle'].tolist()
pandas_output: ['Customer segments not covered by loyalty programs ', 'Customer segments not covered by loyalty programs ', 'Enterprise customers not provided with self service loyalty redemption option', 'Enterpris...

> Running module llm2 with input: 
prompt: Given an input question, synthesize a response from the query results.
Query: What are all the IssueTitle raised in the audit table?

Pandas Instructions (optional):
 df['IssueTitle'].tolist()

Pandas...




llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =      23.12 ms /   201 runs   (    0.12 ms per token,  8691.89 tokens per second)
llama_print_timings: prompt eval time =    3519.53 ms /   171 tokens (   20.58 ms per token,    48.59 tokens per second)
llama_print_timings:        eval time =   49630.55 ms /   200 runs   (  248.15 ms per token,     4.03 tokens per second)
llama_print_timings:       total time =   53459.08 ms /   371 tokens


In [79]:
#Issue summary

print(str(llm.complete(f'can you share a list of only critical issues for higher level leadership from {str(response)}. \
Check if issues can be merged and merge them if they will refer to largely similar issue at a high level. \
Do not share any other explanation text or notes or anything else, only a list of the issues/merged issues and everything should be numbered, if its not numbered drop it')))

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =       9.75 ms /   109 runs   (    0.09 ms per token, 11176.05 tokens per second)
llama_print_timings: prompt eval time =    3687.24 ms /    10 tokens (  368.72 ms per token,     2.71 tokens per second)
llama_print_timings:        eval time =    6235.22 ms /   108 runs   (   57.73 ms per token,    17.32 tokens per second)
llama_print_timings:       total time =    9998.35 ms /   118 tokens


 1. Gaps in loyalty program coverage for various customer segments

2. Lack of self-service options for enterprise customers within loyalty programs

3. Loyalty points expiration issue for churned customers

4. Low enrollment rates for new customers on the bonga loyalty program

5. Payment issues related to license fees for a system under development

6. Reconciliation variances in merchandised redemption payments


In [86]:
response_recom = qp.run(
    query_str="What are all the RecommendationTitle in the audit table?",
)

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?


> Running module input with input: 
query_str: What are all the RecommendationTitle in the audit table?

> Running module pandas_prompt with input: 
query_str: What are all the RecommendationTitle in the audit table?

> Running module llm1 with input: 
prompt: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  ProjectCode                                       ProjectTitle  \
0   ...



Llama.generate: prefix-match hit

llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =       1.64 ms /    16 runs   (    0.10 ms per token,  9779.95 tokens per second)
llama_print_timings: prompt eval time =   16094.84 ms /  1078 tokens (   14.93 ms per token,    66.98 tokens per second)
llama_print_timings:        eval time =    1259.42 ms /    15 runs   (   83.96 ms per token,    11.91 tokens per second)
llama_print_timings:       total time =   17377.65 ms /  1093 tokens
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/pd_llm/lib/python3.9/site-packages/llama_index/query_engine/pandas/output_parser.py", line 54, in default_output_processor
    output_str = str(safe_eval(module_end_str, {"np": np}, local_vars))
  File "/opt/homebrew/anaconda3/envs/pd_llm/lib/python3.9/site-packages/llama_index/exec_utils.py", line 140, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
  File "<string>",

> Running module pandas_output_parser with input: 
input:  ```python
df['RecommendationTitle'].unique()
```

> Running module response_synthesis_prompt with input: 
query_str: What are all the RecommendationTitle in the audit table?
pandas_instructions:  ```python
df['RecommendationTitle'].unique()
```
pandas_output: There was an error running the output as Python code. Error message: name 'p' is not defined

> Running module llm2 with input: 
prompt: Given an input question, synthesize a response from the query results.
Query: What are all the RecommendationTitle in the audit table?

Pandas Instructions (optional):
 ```python
df['RecommendationTit...




llama_print_timings:        load time =   10763.66 ms
llama_print_timings:      sample time =      17.07 ms /   172 runs   (    0.10 ms per token, 10077.93 tokens per second)
llama_print_timings: prompt eval time =    1741.43 ms /    94 tokens (   18.53 ms per token,    53.98 tokens per second)
llama_print_timings:        eval time =   13137.15 ms /   171 runs   (   76.83 ms per token,    13.02 tokens per second)
llama_print_timings:       total time =   15032.88 ms /   265 tokens


In [ ]:

print(str(llm.complete(f'can you share a list of only critical recommendations for higher level leadership from {str(response_recom)}. \
Check if recommendations can be merged and merge them if they will refer to largely similar recommendations at a high level. \
Do not share any other explanation text or notes or anything else, only a list of the issues/merged recommendations and everything should be numbered, if its not numbered drop it')))

In [80]:
df.columns

Index(['ProjectCode', 'ProjectTitle', 'ProjectScore', 'AuditPlanTitle',
       'ProjectQuarter', 'Division', 'ProjectSource', 'ProjectPhase',
       'ProjectScheduledStartDate', 'ProjectScheduledEndDate', 'IssueTitle',
       'RiskRating', 'IssueReleased', 'IssueReleaseDate', 'IssueState',
       'RecommendationTitle', 'ManagementAction', 'DueDate', 'ReviewDueDate',
       'RecommendationDivision', 'RecommendationState', 'TrackingStatus',
       'LastStatusUpdate', 'LastStatusUpdateUser', 'LastStatusUpdateDate',
       'OverdueDescription', 'OverdueStatus', 'OverdueDays', 'Auditor',
       'IssueOwner', 'SeniorManager', 'HOD'],
      dtype='object')

In [85]:
df['RecommendationTitle']

0    Develop and implement loyalty program for home...
1    Develop the enterprise loyalty program to defi...
2    Implement the customer portal for initiation o...
3    Ensure expiry of loyalty value for churned cus...
4    Re-design the kindergarten message and program...
5    Track and ensure delivery of the solution with...
6    Re-design a daily bonga everywhere redemption ...
Name: RecommendationTitle, dtype: object